In [1]:
pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 57.5 MB/s eta 0:00:00m eta 0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs disponibles:", physical_devices)

if len(physical_devices) > 0:
    print("TensorFlow está utilizando la GPU.")
else:
    print("TensorFlow no está utilizando la GPU.")


2024-05-28 18:14:34.160549: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 18:14:34.181046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 18:14:34.181068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 18:14:34.181664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-28 18:14:34.185297: I tensorflow/core/platform/cpu_feature_guar

GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow está utilizando la GPU.


2024-05-28 18:14:34.839134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 18:14:34.859137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 18:14:34.859236: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.layers import TimeDistributed, Bidirectional, LSTM, ConvLSTM2D

def conv_block(input_tensor, num_filters):
    x = TimeDistributed(Conv2D(num_filters, (3, 3), padding='same', activation='elu'))(input_tensor)
    x = TimeDistributed(Conv2D(num_filters, (3, 3), padding='same', activation='elu'))(x)
    return x

def conv_lstm_block(input_tensor, num_filters):
    x = Bidirectional(ConvLSTM2D(num_filters, (3, 3), padding='same', return_sequences=True))(input_tensor)
    return x

def unet_lstm(input_size=(256, 256, 3), n_sequences=10, n_filters=64, n_classes=4):
    inputs = Input((n_sequences, *input_size))

    # Downsample
    c1 = conv_block(inputs, n_filters)
    l1 = conv_lstm_block(c1, n_filters)
    p1 = TimeDistributed(MaxPooling2D((2, 2)))(l1)

    c2 = conv_block(p1, n_filters*2)
    l2 = conv_lstm_block(c2, n_filters*2)
    p2 = TimeDistributed(MaxPooling2D((2, 2)))(l2)

    # Bottleneck
    c3 = conv_block(p2, n_filters*4)
    l3 =  conv_lstm_block(c3, n_filters)

    # Upsample
    u1 = TimeDistributed(UpSampling2D((2, 2)))(l3)
    concat1 = concatenate([u1, l2], axis=4)
    c4 = conv_block(concat1, n_filters*2)
    l4 =  conv_lstm_block(c4, n_filters)

    u2 = TimeDistributed(UpSampling2D((2, 2)))(l4)
    concat2 = concatenate([u2, l1], axis=4)
    c5 = conv_block(concat2, n_filters)
    l5 = conv_lstm_block(c5, n_filters)

    # Output layer
    outputs = TimeDistributed(Conv2D(n_classes, (1, 1), activation='softmax'))(l5)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Create the model
model = unet_lstm()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


2024-06-12 10:57:38.023920: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 10:57:38.392789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 10:57:38.392900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 10:57:38.448456: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 10:57:38.551111: I tensorflow/core/platform/cpu_feature_guar

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 256, 256, 3)]    0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 10, 256, 256, 64)     1792      ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 10, 256, 256, 64)     36928     ['time_distributed[0][0]']    
 stributed)                                                                                       
                                                                                              

In [ ]:
pip install graphviz pydot


In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming your model is defined as 'model'
model = unet_lstm()

# Generate the plot
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True, expand_nested=True)


In [5]:
pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 24.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Input, Conv2D, MaxPooling2D, UpSampling2D, Activation, Dense, Concatenate
from tensorflow_addons.layers import InstanceNormalization, FilterResponseNormalization

"""
GlottisNetV2e (Final version of GlottisNetV2)

Parameters
----------

filters : int, optional
    The number of filters in the first layer.
    The subsequent layers have multiples of this filter number. Default: 16

layers : int, optional
    The number of encoding and decoding layers. Default: 4.

Returns
-------

Keras Model
    Using different decoders for prediction maps of anterior and posterior points and segmentation
    U-Net structure in tensorflow.keras with 2 outputs
    First output: Prediction maps of of anterior and posterior point (2 channels)
    Second output: Segmentation map"""

def Decoder(input_tensor, to_concat, name = 'decoder_', layers = 4, filters = 16):
    x  = input_tensor

    # Decoding path for prediction maps of anterior and posterior points
    for step, filter_factor in enumerate(np.arange(layers)[::-1]):

        # First convolution in layer followed by instance normalization and activation function
        x = TimeDistributed(Conv2D(filters * (2 ** filter_factor), (3, 3), use_bias = False, padding = 'same', \
                         strides = 1, kernel_initializer = 'he_uniform', name = name + "conv1" + str(step))(x))
        x = Bidirectional(ConvLSTM2D(filters, (3, 3), padding='same', return_sequences=True))(input_tensor)
        x = InstanceNormalization()(x)
        x = Activation('relu')(x)

        # Umsampling and concatenation with results of encoding path
        x = TimeDistributed(UpSampling2D(size = (2, 2), name = name + "UpSampling" + str(step))(x))
        x = Concatenate()([x, to_concat[::-1][step]])

        # Second convolution followed by batch normalization and activation function
        x = TimeDistributed(Conv2D(filters * (2 ** filter_factor), (3, 3), use_bias = False, padding = 'same', \
                         strides = 1, kernel_initializer = 'he_uniform', name = name + "conv2" + str(step))(x))
        x = Bidirectional(ConvLSTM2D(filters, (3, 3), padding='same', return_sequences=True))(input_tensor)

        x = InstanceNormalization()(x)
        x = Activation('relu')(x)


    # Last decoding step
    x =TimeDistributed( Conv2D(filters * (2 ** filter_factor), (3, 3), use_bias = False, padding = 'same', \
                     strides = 1, kernel_initializer = 'he_uniform', name = name + "last_decoding_step" + str(step))(x))
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    return x


def glottisnetV2_e(filters = 16, layers = 4, input_size = (512, 256, 1)):

    in_layer = Input(input_size)
    x = in_layer

    # save layers to concat
    to_concat = []

    # Encoding path
    for step in range(layers):

        # Per layer apply two convolutions each followed by instance normalization and an activation function
        x = TimeDistributed(Conv2D(filters * (2 ** step), (3, 3), use_bias = False, padding = 'same', strides = 1, \
                   kernel_initializer = 'he_uniform')(x))
        x = Bidirectional(ConvLSTM2D(filters, (3, 3), padding='same', return_sequences=True))(input_tensor)

        x = InstanceNormalization()(x)
        x = Activation('relu')(x)

        x = TimeDistributed(Conv2D(filters * (2 ** step), (3, 3), use_bias = False, padding = 'same', strides = 1, \
                   kernel_initializer = 'he_uniform')(x))
        x = InstanceNormalization()(x)
        x = Activation('relu')(x)

        # Append input to list to use it for concatenation in decoding path
        to_concat.append(x)
        x =TimeDistributed(MaxPooling2D(pool_size = (2, 2))(x))

    # Last convolution (without MaxPooling) in latent space
    x = TimeDistributed(Conv2D(filters * (2 ** (step + 1)), (3, 3), use_bias = False, padding = 'same', strides = 1, \
               kernel_initializer = 'he_uniform')(x))
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)

    # Add two decoders for
    upconv_seg = Decoder(x, to_concat, name = "decode_seg", layers= layers, filters = filters)
    # Output maps
    # 1x1 convolution to create 3 output maps (segmentation, prediction map of anterior point, prediction map of posterior point)
    out_seg = TimeDistributed(Conv2D(1, (1, 1), use_bias = False, padding = "same", activation = 'sigmoid', strides = 1, \
                     kernel_initializer = 'glorot_uniform', name = 'seg')(upconv_seg))


   # Create model
    model = Model(in_layer, [out_ap, out_seg])

    return model


/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import numpy as np
from random import shuffle
import logging
from glob import glob
from PIL import Image

from random import shuffle
from glob import glob

class SequenceDataProvider:
    def __init__(self, data_dir, sequence_length, image_suffix, label_suffix, input_size, n_classes, shuffle_data=True, max_sequences=None):
        self._data_dir = data_dir
        self._sequence_length = sequence_length
        self._image_suffix = image_suffix
        self._label_suffix = label_suffix
        self._input_size = input_size
        self._n_classes = n_classes
        self._shuffle_data = shuffle_data
        self._max_sequences = max_sequences
        self._sequences = self._load_sequences()

    def dataset_length(self):
        # Devuelve el número total de secuencias cargadas
        return len(self._sequences)

    def _load_sequences(self):
        image_paths = sorted(glob(f"{self._data_dir}/*{self._image_suffix}"))
        label_paths = sorted(glob(f"{self._data_dir}/*{self._label_suffix}"))

        sequences = [
            list(zip(image_paths[i:i+self._sequence_length], label_paths[i:i+self._sequence_length]))
            for i in range(0, len(image_paths) - self._sequence_length + 1, self._sequence_length)
        ]

        if self._max_sequences:
            sequences = sequences[:self._max_sequences]

        if self._shuffle_data:
            shuffle(sequences)
        print(f"Loaded {len(sequences)} sequences.")
        return sequences

    def _process_image(self, image_path):
        image = Image.open(image_path)
        image = image.resize(self._input_size[:2])  # Assuming input_size is (height, width, channels)
        return np.array(image)

    def _process_label(self, label_path):
        label = Image.open(label_path)
        label = label.resize(self._input_size[:2])
        label = np.array(label, dtype=np.int32)  # Asegúrate de que sea entero para usar como índices

        # Convertir la etiqueta a formato one-hot
        one_hot_label = np.eye(self._n_classes)[label]
        return one_hot_label

    def next_batch(self, batch_size=1):
        batch_images = []
        batch_labels = []
        for _ in range(batch_size):
            if len(self._sequences) == 0:
                self._sequences = self._load_sequences()

            sequence = self._sequences.pop(0)
            sequence_images = []
            sequence_labels = []

            for image_path, label_path in sequence:
                image = self._process_image(image_path)
                label = self._process_label(label_path)
                sequence_images.append(image)
                sequence_labels.append(label)

            batch_images.append(sequence_images)
            batch_labels.append(sequence_labels)
        print(np.array(batch_labels).shape)

        batch_images = np.array(batch_images).reshape(batch_size, self._sequence_length, *self._input_size)
        # Asegúrate de ajustar el reshape de batch_labels para que coincida con las dimensiones esperadas por tu modelo
        batch_labels = np.array(batch_labels).reshape(batch_size, self._sequence_length, *self._input_size[:2], self._n_classes)

        return batch_images, batch_labels

# Asegúrate de definir input_size y n_classes acorde a tu modelo
input_size = (256, 256, 3)  # Ejemplo de tamaño de entrada
n_classes = 4  # Ejemplo de número de clases

data_provider = SequenceDataProvider(
    data_dir="/content/drive/MyDrive/Glottis/dataset/dataset/train",
    sequence_length=10,
    image_suffix="_rgb.png",
    label_suffix="_mask.png",
    input_size=(256, 256, 3),
    n_classes=4,
    shuffle_data=True,
    max_sequences=10
)



Loaded 10 sequences.


In [ ]:
epochs = 10  # Número de épocas para el entrenamiento
batch_size = 2  # Tamaño del lote
def data_generator(batch_size):
    while True:
        batch_images, batch_labels = data_provider.next_batch(batch_size)
        yield batch_images, batch_labels

# Crear el generador
train_generator = data_generator(batch_size)

# Calcular el número de pasos por época
steps_per_epoch = data_provider.dataset_length() // batch_size

# Entrenar el modelo
model.fit(x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=epochs)



(2, 10, 256, 256, 4)
Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node model/bidirectional/backward_conv_lstm2d/TensorArrayV2Stack/TensorListStack defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py